<a href="https://colab.research.google.com/github/KauaHenSilva/python_game_price_prediction/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recuperar arquivos

In [12]:
import os
tipo_de_recuperacao = "Ambos" #@param ["Download", "Drive", "Ambos"]
def repostirio_para_download():
  if os.path.exists('python_download_file_drive'):
    !cd python_download_file_drive && git pull
  else:
    !git clone https://github.com/KauaHenSilva/python_download_file_drive
  %pip install --upgrade --quiet -r python_download_file_drive/requirements.txt

## Recuperação por download

In [13]:
if tipo_de_recuperacao in ["Download", "Ambos"]:
  from google.colab import auth
  from googleapiclient.discovery import build
  auth.authenticate_user()
  repostirio_para_download()
  !python "python_download_file_drive/main.py" "https://drive.google.com/drive/folders/19BjVYpKOsLwjsqH5LMMZ_RjYdVUy0_HJ?usp=drive_link" arquivos

Already up to date.
Baixando arquivos: 100% 1/1 [00:00<00:00, 19065.02arquivo/s]


## Recuperação por drive

In [14]:
# @markdown #### Caso escolheu o tipo de recuperação drive/ambos e não tenha os arquivos, faça um atalho desse drive:
# @markdown > https://drive.google.com/drive/folders/19BjVYpKOsLwjsqH5LMMZ_RjYdVUy0_HJ?usp=drive_link
path_atalho_drive = "/content/drive/MyDrive/Projetos Do GitHub/python_game_price_prediction" #@param {type:"string"}

In [15]:
if tipo_de_recuperacao in ["Drive", "Ambos"]:
  from google.colab import drive
  drive.mount('/content/drive')
  if not os.path.exists(path_atalho_drive):
    raise Exception("O caminho especificado não existe no drive.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Referenciar arquivo no drive ou download

In [16]:
def get_referencia_valida(path):
    path_download = os.path.join("/content/arquivos", path)
    path_drive = os.path.join(path_atalho_drive, path)

    bool_download = os.path.exists(path_download)
    bool_drive = os.path.exists(path_drive)

    if not bool_download and not bool_drive:
        raise Exception(f"Arquivo não encontrado em nenhum dos caminhos (Download ou Drive). Caminhos: {path_download}, {path_drive}")

    if not bool_drive and bool_download:
        print("[INFO] Arquivo encontrado no download, mas não no drive.")
        return path_download

    if bool_drive and not bool_download:
        print("[INFO] Arquivo encontrado no drive, mas não no download.")
        return path_drive

    return path_download


# Games

In [17]:
!pip install tensorflow==2.16.1 --quiet

In [18]:
import tensorflow
import pandas as pd
import sklearn

In [19]:
tensorflow.__version__, pd.__version__, sklearn.__version__

('2.16.1', '2.2.2', '1.6.1')

In [58]:
from tensorflow.keras.layers import Dense, Activation, Input, Dropout
from tensorflow.keras.models import Model
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error

In [21]:
path_base = get_referencia_valida('games.csv')
base = pd.read_csv(path_base)
base

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16714,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,Tecmo Koei,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16715,LMA Manager 2007,X360,2006.0,Sports,Codemasters,0.00,0.01,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16716,Haitaka no Psychedelica,PSV,2016.0,Adventure,Idea Factory,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16717,Spirits & Spells,GBA,2003.0,Platform,Wanadoo,0.01,0.00,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
base = base.drop("Other_Sales", axis=1)
base = base.drop("Global_Sales", axis=1)
base = base.drop("Developer", axis=1)
base

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,76.0,51.0,8,322.0,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,82.0,73.0,8.3,709.0,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,80.0,73.0,8,192.0,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16714,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,Tecmo Koei,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN
16715,LMA Manager 2007,X360,2006.0,Sports,Codemasters,0.00,0.01,0.00,NaN,NaN,NaN,NaN,NaN
16716,Haitaka no Psychedelica,PSV,2016.0,Adventure,Idea Factory,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN
16717,Spirits & Spells,GBA,2003.0,Platform,Wanadoo,0.01,0.00,0.00,NaN,NaN,NaN,NaN,NaN


In [25]:
base.isnull().sum()

,0
Name,2
Platform,0
Year_of_Release,269
Genre,2
Publisher,54
NA_Sales,0
EU_Sales,0
JP_Sales,0
Critic_Score,8582
Critic_Count,8582


In [26]:
base = base.dropna(axis=0)
base

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,76.0,51.0,8,322.0,E
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,82.0,73.0,8.3,709.0,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,80.0,73.0,8,192.0,E
6,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.28,9.14,6.50,89.0,65.0,8.5,431.0,E
7,Wii Play,Wii,2006.0,Misc,Nintendo,13.96,9.18,2.93,58.0,41.0,6.6,129.0,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16667,E.T. The Extra-Terrestrial,GBA,2001.0,Action,NewKidCo,0.01,0.00,0.00,46.0,4.0,2.4,21.0,E
16677,Mortal Kombat: Deadly Alliance,GBA,2002.0,Fighting,Midway Games,0.01,0.00,0.00,81.0,12.0,8.8,9.0,M
16696,Metal Gear Solid V: Ground Zeroes,PC,2014.0,Action,Konami Digital Entertainment,0.00,0.01,0.00,80.0,20.0,7.6,412.0,M
16700,Breach,PC,2011.0,Shooter,Destineer,0.01,0.00,0.00,61.0,12.0,5.8,43.0,T


In [27]:
base.isnull().sum()

,0
Name,0
Platform,0
Year_of_Release,0
Genre,0
Publisher,0
NA_Sales,0
EU_Sales,0
JP_Sales,0
Critic_Score,0
Critic_Count,0


In [29]:
base['Name'].value_counts()

,count
Name,
Need for Speed: Most Wanted,8
Madden NFL 07,8
LEGO Star Wars II: The Original Trilogy,8
The Sims 2,7
Terraria,7
...,...
Castlevania: Portrait of Ruin,1
Suzuki TT Superbikes,1
Rumble Roses,1


In [30]:
base = base.drop("Name", axis=1)
base

,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,76.0,51.0,8,322.0,E
2,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,82.0,73.0,8.3,709.0,E
3,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,80.0,73.0,8,192.0,E
6,DS,2006.0,Platform,Nintendo,11.28,9.14,6.50,89.0,65.0,8.5,431.0,E
7,Wii,2006.0,Misc,Nintendo,13.96,9.18,2.93,58.0,41.0,6.6,129.0,E
...,...,...,...,...,...,...,...,...,...,...,...,...
16667,GBA,2001.0,Action,NewKidCo,0.01,0.00,0.00,46.0,4.0,2.4,21.0,E
16677,GBA,2002.0,Fighting,Midway Games,0.01,0.00,0.00,81.0,12.0,8.8,9.0,M
16696,PC,2014.0,Action,Konami Digital Entertainment,0.00,0.01,0.00,80.0,20.0,7.6,412.0,M
16700,PC,2011.0,Shooter,Destineer,0.01,0.00,0.00,61.0,12.0,5.8,43.0,T


In [39]:
x = base.iloc[:, [0,1,2,3,7,8,9,10,11]].values
x

array([['Wii', 2006.0, 'Sports', ..., '8', 322.0, 'E'],
       ['Wii', 2008.0, 'Racing', ..., '8.3', 709.0, 'E'],
       ['Wii', 2009.0, 'Sports', ..., '8', 192.0, 'E'],
       ...,
       ['PC', 2014.0, 'Action', ..., '7.6', 412.0, 'M'],
       ['PC', 2011.0, 'Shooter', ..., '5.8', 43.0, 'T'],
       ['PC', 2011.0, 'Strategy', ..., '7.2', 13.0, 'E10+']], dtype=object)

In [36]:
y_na = base.iloc[:, [4]].values
y_eu = base.iloc[:, [5]].values
y_jp = base.iloc[:, [6]].values
y_na, y_eu, y_jp

(array([[4.136e+01],
        [1.568e+01],
        [1.561e+01],
        ...,
        [0.000e+00],
        [1.000e-02],
        [0.000e+00]]),
 array([[2.896e+01],
        [1.276e+01],
        [1.093e+01],
        ...,
        [1.000e-02],
        [0.000e+00],
        [1.000e-02]]),
 array([[3.77],
        [3.79],
        [3.28],
        ...,
        [0.  ],
        [0.  ],
        [0.  ]]))

In [40]:
x[0]

array(['Wii', 2006.0, 'Sports', 'Nintendo', 76.0, 51.0, '8', 322.0, 'E'],
      dtype=object)

In [41]:
onehotencoder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [0, 2, 3, 8])], remainder='passthrough')
X = onehotencoder.fit_transform(x).toarray()
X, X.shape

(array([[  0. ,   0. ,   0. , ...,  51. ,   8. , 322. ],
        [  0. ,   0. ,   0. , ...,  73. ,   8.3, 709. ],
        [  0. ,   0. ,   0. , ...,  73. ,   8. , 192. ],
        ...,
        [  0. ,   0. ,   0. , ...,  20. ,   7.6, 412. ],
        [  0. ,   0. ,   0. , ...,  12. ,   5.8,  43. ],
        [  0. ,   0. ,   0. , ...,  12. ,   7.2,  13. ]]),
 (6825, 303))

In [44]:
camada_entrada = Input(shape=(303,))
camada_oculta_1 = Dense(units=153, activation='relu')(camada_entrada) # (303 + 3) / 2
camada_oculta_2 = Dense(units=153, activation='relu')(camada_oculta_1)
camada_saida_1 = Dense(units=1, activation='linear')(camada_oculta_2)
camada_saida_2 = Dense(units=1, activation='linear')(camada_oculta_2)
camada_saida_3 = Dense(units=1, activation='linear')(camada_oculta_2)

In [46]:
net = Model(camada_entrada, [camada_saida_1, camada_saida_2, camada_saida_3] )

In [48]:
net.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 303)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 153)            │         46,512 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 153)            │         23,562 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 1)              │            154 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 1)              │            154 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 1)              │            154 │ dense_1[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 70,536 (275.53 KB)

 Trainable params: 70,536 (275.53 KB)

 Non-trainable params: 0 (0.00 B)

In [49]:
net.compile(optimizer='adam', loss='mse')

In [50]:
net.fit(x = X, y=[y_na, y_eu, y_jp], epochs=500, batch_size=100)

Epoch 1/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - dense_2_loss: 184.4983 - dense_3_loss: 528.0406 - dense_4_loss: 106.8621 - loss: 819.5295
Epoch 2/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - dense_2_loss: 5.2030 - dense_3_loss: 8.3228 - dense_4_loss: 10.1041 - loss: 23.6331
Epoch 3/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - dense_2_loss: 1.2043 - dense_3_loss: 0.7789 - dense_4_loss: 1.2364 - loss: 3.2198
Epoch 4/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - dense_2_loss: 1.1937 - dense_3_loss: 0.7289 - dense_4_loss: 0.7008 - loss: 2.6225
Epoch 5/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - dense_2_loss: 2.8408 - dense_3_loss: 3.8960 - dense_4_loss: 10.3526 - loss: 17.0807
Epoch 6/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - dense_2_loss: 4.2098 - dense_3_loss: 8.4353 - dense_4_loss: 28.4390 - loss: 41.0798
Epoch 7/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - dense_2_loss: 9.0653 - dense_3_loss: 6.1998 - dense_4_loss: 15.9172 - loss: 31.1859
Epoch 8/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 

In [54]:
result_na, result_eu, result_jp = net.predict(X)
result_na, result_eu, result_jp

214/214 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


(array([[10.257277  ],
        [11.497718  ],
        [ 3.8779674 ],
        ...,
        [-0.2067259 ],
        [ 0.11240786],
        [ 0.04787451]], dtype=float32),
 array([[ 7.4797049e+00],
        [ 8.2913513e+00],
        [ 2.6541719e+00],
        ...,
        [-2.2417992e-02],
        [ 7.2608888e-02],
        [-6.3338876e-04]], dtype=float32),
 array([[ 2.3916664 ],
        [ 2.694651  ],
        [ 0.7343235 ],
        ...,
        [-0.06837144],
        [ 0.01728734],
        [-0.04651061]], dtype=float32))

In [55]:
result_na, y_na, result_na.mean(), y_na.mean()

(array([[10.257277  ],
        [11.497718  ],
        [ 3.8779674 ],
        ...,
        [-0.2067259 ],
        [ 0.11240786],
        [ 0.04787451]], dtype=float32),
 array([[4.136e+01],
        [1.568e+01],
        [1.561e+01],
        ...,
        [0.000e+00],
        [1.000e-02],
        [0.000e+00]]),
 0.38228393,
 0.3944835164835165)

In [56]:
result_eu, y_eu, result_eu.mean(), y_eu.mean()


(array([[ 7.4797049e+00],
        [ 8.2913513e+00],
        [ 2.6541719e+00],
        ...,
        [-2.2417992e-02],
        [ 7.2608888e-02],
        [-6.3338876e-04]], dtype=float32),
 array([[2.896e+01],
        [1.276e+01],
        [1.093e+01],
        ...,
        [1.000e-02],
        [0.000e+00],
        [1.000e-02]]),
 0.21822391,
 0.23608937728937732)

In [57]:
result_jp, y_jp, result_jp.mean(), y_jp.mean()


(array([[ 2.3916664 ],
        [ 2.694651  ],
        [ 0.7343235 ],
        ...,
        [-0.06837144],
        [ 0.01728734],
        [-0.04651061]], dtype=float32),
 array([[3.77],
        [3.79],
        [3.28],
        ...,
        [0.  ],
        [0.  ],
        [0.  ]]),
 0.034124725,
 0.06415824175824175)

In [60]:
mean_absolute_error(y_na, result_na), mean_absolute_error(y_eu, result_eu), mean_absolute_error(y_jp, result_jp)

(0.2670239032310444, 0.18483109039919712, 0.07684694903447077)